In [ ]:
# install the kaggle library
! pip install kaggle

In [ ]:
import os
os.makedirs('/root/.kaggle', exist_ok=True)
!cp kaggle.json /root/.kaggle/



cp: cannot stat 'kaggle.json': No such file or directory


In [ ]:
# configuration the path of kaggle.json file

!chmod 600 /root/.kaggle/kaggle.json


chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


importing the dog vs cat dataset from kaggle

In [ ]:
#kaggle api
!kaggle competitions download -c dogs-vs-cats



Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python3.10/dist-packages/kaggle/__init__.py", line 7, in <module>
    api.authenticate()
  File "/usr/local/lib/python3.10/dist-packages/kaggle/api/kaggle_api_extended.py", line 398, in authenticate
    raise IOError('Could not find {}. Make sure it\'s located in'
OSError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [ ]:
# extracting the compressed datset
from zipfile import ZipFile

dataset = '/content/dogs-vs-cats.zip'
with ZipFile(dataset , 'r') as zip :
  zip.extractall()
  print ('The dataset is extracted')

FileNotFoundError: [Errno 2] No such file or directory: '/content/dogs-vs-cats.zip'

In [ ]:
# extracting the compressed datset
from zipfile import ZipFile

dataset = '/content/train.zip'
with ZipFile(dataset , 'r') as zip :
  zip.extractall()
  print ('The dataset is extracted')

In [ ]:
import os
# counting the number of files in train folder
path, dirs , files = next(os.walk('/content/train'))
file_count= len(files)
print ('number of images', file_count)

printing the name of images

In [ ]:
file_names = os.listdir('/content/train/')
print (file_names)

IMport all the dependencies

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
from google.colab.patches import cv2_imshow

Displaying the images of thre dog abnd cat


In [ ]:
# display the dog image
img = mpimg.imread('/content/train/dog.6242.jpg')
imgplot = plt.imshow(img)
plt.show()

In [ ]:
# display the cat image
img = mpimg.imread('/content/train/cat.9107.jpg')
imgplot = plt.imshow(img)
plt.show()

In [ ]:
file_names = os.listdir('/content/train/')

for i in range (5):
  name = file_names[i]
  print (name[0:3])


In [ ]:
file_names = os.listdir('/content/train/')

dog_count=0
cat_count=0

for img_file in file_names:

  name = img_file[0:3]

  if name == 'dog':
    dog_count+=1

  else :
    cat_count+=1
print('number of dog images = ', dog_count)
print('number of cat images = ', cat_count)


Resizing all the images

In [ ]:
# creating the directory for resized images
os.mkdir('/content/image resized')

In [ ]:
original_folder = '/content/train/'
resized_folder = '/content/image resized/'

for i in range (2000):

  filename = os.listdir(original_folder)[i]
  img_path = original_folder+filename

  img = Image.open(img_path)
  img = img.resize((224 , 224))
  img = img.convert('RGB')

  newImgPath = resized_folder+filename
  img.save(newImgPath)

In [ ]:
# display the dog image
img = mpimg.imread('/content/image resized/dog.6242.jpg')
imgplot = plt.imshow(img)
plt.show()

In [ ]:
# display the cat image
img = mpimg.imread('/content/image resized/cat.9107.jpg')
imgplot = plt.imshow(img)
plt.show()

Creating labels for resize images of dogs and cats

cat--> 0
dog--> 1

In [ ]:
# creating the for loop to assign labels
filenames = os.listdir('/content/image resized/')


labels =[]

for i in range (2000):
  file_name = filenames[i]
  label = file_name[0:3]

  if label == 'dog':
    labels.append(1)
  else:
    labels.append(0)

In [ ]:
print(filenames[0:5])
print(len(labels))


In [ ]:
# counting the images of dogs and cats out of 2000 images
values, counts = np.unique(labels, return_counts = True)
print (values)
print(counts)


Converting all the resized images to numpy array

In [ ]:
import cv2
import glob

In [ ]:
image_directory = '/content/image resized/'
image_extension = ['png','jpg']

files = []

[files.extend(glob.glob(image_directory + "*." + e )) for e in image_extension]

dog_cat_images = np.asarray([cv2.imread(file) for file in files])


In [ ]:
print(dog_cat_images)

In [ ]:
type(dog_cat_images)

In [ ]:
print (dog_cat_images.shape)

In [ ]:
X =dog_cat_images
Y = np.asarray(labels)

Train Test split

In [ ]:
X_train, X_test , Y_train ,Y_test = train_test_split(X,Y, test_size=0.2, random_state=2)

In [ ]:
print (X.shape, X_train.shape, X_test.shape)

we add 1600 --> training images

400 --> test images

In [ ]:
# scaling hte data
X_train_scaled = X_train/255

X_test_scaled = X_test/255

In [ ]:
print(X_train_scaled)

building the neaural network

In [ ]:
import tensorflow as tf
import tensorflow_hub as hub

In [ ]:
mobilenet_model ='https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4'

pretrained_model = hub.KerasLayer(mobilenet_model, input_shape=(224,224,3), trainable=False)

In [ ]:
num_of_classes = 2

model = tf.keras.Sequential([

    pretrained_model,
    tf.keras.layers.Dense(num_of_classes)

])

model.summary()
Model: "sequential"

In [ ]:
model.compile(

      optimizer = 'adam',
      loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics = ['acc']


)

In [ ]:
model.fit(X_train_scaled, Y_train, epochs=5)

In [ ]:
score, acc = model.evaluate(X_test_scaled,Y_test)
print('test loss = ', score)
print ('test Accuracy = ', acc)


predictive system


In [ ]:
from google.colab.patches import cv2_imshow
import cv2
import numpy as np

input_image_path = input('path of the image to be predicted: ')

input_image = cv2.imread(input_image_path)

cv2_imshow(input_image)

input_image_resize = cv2.resize(input_image, (224,224))

input_image_scaled = input_image_resize / 255.0

image_reshaped = np.reshape(input_image_scaled, [1,224,224,3])

input_prediction = model.predict(image_reshaped)

print(input_prediction)

input_pred_label = np.argmax(input_prediction)

print(input_pred_label)

if input_pred_label == 0 :
    print('The image represents a cat.')
else :
    print('The image represents a dog.')
